In [1]:
# Creiamo un regressore logistico per classificare le cifre MNIST usando
# a pieno TF 1 e il concetto di grafo delle operazioni e di sessione

import tensorflow.compat.v1 as tf # evitiamo le continue richieste di usare tf.compat.v1.*
from tensorflow import keras
import numpy as np
from tqdm import trange
import os
# Non abilitiamo la eager execution: useremo il grafo di computazione e la tf.Session
# tf.compat.v1.enable_eager_execution()

# log dei messaggi solo  per gli errori
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# percorso di salvataggio dei pesi e dei log per tensorboard
model_dir = '/home/rpirrone/tf1.15/mnist_logistic_regression'

save_path = os.path.join(model_dir,'weights','epoch_')

In [2]:
# Importiamo il dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Creiamo la struttura del data set per l'addestramento del modello
def process_input_data(x_train, y_train, x_test):
    
    # stimiamo il numero delle classi e delle feature per creare un vettore monodimensionale

    num_classes = y_train.max()+1 # le classi hanno etichette numeriche da 0 a num_classes - 1

    num_features = 1
    
    for dim in x_train[0].shape:
        num_features *= dim     # le feature sono il prodotto delle dimensioni dell'immagine campione
        
    # Convertiamo in float32.

    x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

    # Convertiamo le immagini con shape (28,28) in un vettore monodimensionale di 784 elementi (Flattening)

    x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])

    # Creiamo l'embedding numerico ovvero la rappresentazione come reali in [0,1]

    x_train, x_test = x_train / 255., x_test / 255.
        
    # il vero Dataset TF è ottenuto per trasformazione di una serie di 'slice' di un tensore
    # ovvero una serie di immagini in una sequenza organizzata come una matrice 3D
    train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))
    
    return (num_classes, num_features, train_data, x_train, x_test)


In [3]:
def train_logistic_regression(x_train, y_train, x_test, y_test,\
    shuffle=50000,\
    epochs=100,\
    save_steps=10,\
    learning_rate=0.01,\
    batch_size=128,\
    momentum=0.9,\
    nesterov=True):
    
    #
    # Pipeline di costruzione del data set
    #
    print('Building the data set ...',end=' ')
    
    # Eseguiamo il preprocessing del data set su CPU e lasciamo il reesto della computazione su GPU
    with tf.device('CPU:0'):
        num_classes, num_features, train_data, x_train, x_test = process_input_data(x_train, y_train, x_test)
        
        # Processing del data set: ripetizione indefinita dei campioni, shuffling e impostazione della dimensione
        # dei batch; impostazione del numero di batch già caricati in memoria quando si itera sul data set
        train_data = (train_data.repeat()\
                                .shuffle(shuffle)\
                                .batch(batch_size)\
                                .prefetch(tf.data.experimental.AUTOTUNE))

    # Creiamo l'iteratore sul data set per scorrere i batch
    # inisieme alla sua operazione di next e all'inizializzatore
    iterator = train_data.make_initializable_iterator()
    next_element = iterator.get_next()
    iterator_init_op = iterator.initializer
    
    print('Done!\n')
    
    #
    # Costruzione del modello e delle metriche di valutazione
    # 
    print('Building the model graph ...',end=' ')
    
    # definiamo il nodo di bias inizializzato a 0
    b = tf.Variable(tf.zeros([num_classes]), name='bias')

    # definiamo il nodo della matrice pesi 'W' inizializzata randomicamente tra -1 e 1
    W = tf.Variable(tf.random_uniform((num_features, num_classes),minval=-1.0,maxval=1.0), name='weight')
    
    # Placeholder per il batch di ingresso e delle etichette di uscita
    x = tf.placeholder(tf.float32, shape=(None,num_features))
    y = tf.placeholder(tf.int64, shape=(None,))
    
    # Inizializzazione delle variabili globali del grafo
    global_init_op = tf.global_variables_initializer()
    
    # predizioni della regressione logistica: log-probabilità non scalate 
    logit = tf.matmul(x, W) + b
    
    # loss per singolo batch: cross-entropia multi classe
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.stop_gradient(tf.one_hot(y,depth=num_classes))\
                                                                    ,logits=logit))
    
    # Definizione delle metriche
    with (tf.variable_scope('metrics')):
        mean_acc, mean_acc_op = tf.metrics.accuracy(labels=tf.argmax(tf.one_hot(y,depth=num_classes),1),\
                                                    predictions=tf.argmax(tf.nn.softmax(logit),1))
        mean_loss, mean_loss_op = tf.metrics.mean(loss)

    # Operazione di aggiornamento delle metriche
    update_metrics_op = tf.group(*[mean_acc_op, mean_loss_op])

    # Inizializzazione delle variabili locali delle metriche
    metric_variables = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="metrics")
    metrics_init_op = tf.variables_initializer(metric_variables)

    print('Done!\n')

    #
    # Ottimizzatore, step di addestramento e relativa inizializzazione
    #
    print('Setting up the optimizer ...',end=' ')
    
    optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,use_nesterov=nesterov,momentum=momentum)
    train_op = optimizer.minimize(loss)
    train_init_op = tf.variables_initializer(optimizer.variables())
    
    print('Done!\n')
    
    #
    # Saver del modello e summary per tensorboard
    #
    print('Creating saver and summaries ...',end=' ')
    
    # definizione dei summary
    tf.summary.scalar('loss', mean_loss)
    tf.summary.scalar('accuracy', mean_acc)
    
    summary_op = tf.summary.merge_all()
    
    # Saver
    saver = tf.train.Saver()
    
    print('Done!\n')
    
    #
    # Addestramento e valutazione del modello
    #
    
    print('---- Start training ----\n')
    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
        
        # Creazione dei writer per tensorboard
        train_writer = tf.summary.FileWriter(os.path.join(model_dir, 'train_summaries'), sess.graph)
        eval_writer = tf.summary.FileWriter(os.path.join(model_dir, 'eval_summaries'), sess.graph)

        # Definizione del numero di batch arrotondato all'intero successivo
        num_batches = x_train.shape[0]//batch_size + 1
        
        print('Global init operations ...',end=' ')
        
        # Operazioni di inizializzazione globali
        sess.run([global_init_op,train_init_op])
        
        print('Done!\n')
               
        for epoch in range(1,epochs + 1):

            # Progress bar
            t = trange(num_batches)
            t.set_description(desc='Epoch {:4d}'.format(epoch))
            
            # Inizializzazione dell'iteratore sul data set e delle metriche di valutazione del training
            sess.run([iterator_init_op,metrics_init_op])
            
            for _ in t:
                
                # prendiamo un batch
                batch_x, batch_y = sess.run(next_element)
                
                # addestriamo, aggiorniamo le metriche e calcoliamo la loss
                _, _, loss_val = sess.run([train_op, update_metrics_op, loss],feed_dict={x: batch_x, y: batch_y})
                
            # Salvataggio del modello ogni save_steps epoche
            if epoch % save_steps == 0:
                saver.save(sess,save_path,global_step=epoch)
                
            # Valutazione dell'epoca di addestramento
            mean_acc_val, mean_loss_val = sess.run([mean_acc, mean_loss])
            out_string = '----Train metrics: accuracy: {:5.3f}; loss: {:5.3f}\n'.format(mean_acc_val, mean_loss_val)
            
            # salvataggio per tensorboard
            summ = sess.run(summary_op)
            train_writer.add_summary(summ,global_step=epoch)
            
            # Valutazione sul test set
            sess.run(metrics_init_op)
            
            sess.run(update_metrics_op,feed_dict={x: x_test, y: y_test})
            mean_acc_val, mean_loss_val = sess.run([mean_acc, mean_loss])
            out_string += '----Eval. metrics: accuracy: {:5.3f}; loss: {:5.3f}\n'.format(mean_acc_val, mean_loss_val)
            
            # salvataggio per tensorboard
            summ = sess.run(summary_op)
            eval_writer.add_summary(summ,global_step=epoch)
            
            print(out_string)

In [4]:
train_logistic_regression(x_train, y_train, x_test, y_test)

Building the data set ... WARNING:tensorflow:From /tmp/ipykernel_23045/4184321682.py:28: DatasetV1.make_initializable_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
Done!

Building the model graph ... Done!

Setting up the optimizer ... Done!

Creating saver and summaries ... Done!

---- Start training ----

Global init operations ... Done!



Epoch    1: 100%|██████████| 469/469 [00:04<00:00, 107.62it/s]


----Train metrics: accuracy: 0.642; loss: 1.596
----Eval. metrics: accuracy: 0.801; loss: 0.726



Epoch    2: 100%|██████████| 469/469 [00:01<00:00, 244.06it/s]


----Train metrics: accuracy: 0.817; loss: 0.658
----Eval. metrics: accuracy: 0.843; loss: 0.561



...

----Train metrics: accuracy: 0.930; loss: 0.255
----Eval. metrics: accuracy: 0.923; loss: 0.281



Epoch   99: 100%|██████████| 469/469 [00:01<00:00, 245.58it/s]


----Train metrics: accuracy: 0.930; loss: 0.255
----Eval. metrics: accuracy: 0.923; loss: 0.281



Epoch  100: 100%|██████████| 469/469 [00:01<00:00, 239.42it/s]


----Train metrics: accuracy: 0.930; loss: 0.252
----Eval. metrics: accuracy: 0.923; loss: 0.281

